In [2]:

from xbbg import blp
import pandas as pd
import numpy as np

SyntaxError: invalid syntax (2549843542.py, line 1)

In [15]:
#Automate to get tickers you need-Note it is setup so that you could add years, country and code as and when new variables are needed. 
quarter= ["Q1","Q2","Q3","Q4"]
year = [21,22,23] #needs to be updated for new year values
country= ["US","CA","GB","AU","CN","IN","JP","EU"]
ind =["Index"]
code = ["ECGQ","ECPI","ECCC","ECUP","ECCB","BYXY"]

In [16]:
#generate dataframe with index tickers
index= []
for co in code:
    for c in country:
        for q in quarter:
            for y in year:
                for i in ind:
                    index.append(co+c+" "+q+str(y)+" "+i)

In [17]:
index=pd.DataFrame(index)
data= index[0]
index

,0
0,ECGQUS Q121 Index
1,ECGQUS Q122 Index
2,ECGQUS Q123 Index
3,ECGQUS Q221 Index
4,ECGQUS Q222 Index
...,...
571,BYXYEU Q322 Index
572,BYXYEU Q323 Index
573,BYXYEU Q421 Index
574,BYXYEU Q422 Index


In [5]:
#Pull data using bdp and at this stage this is PX last 
df = blp.bdp(data,'PX_LAST')

In [6]:
df= pd.DataFrame(df)
df = df.reset_index()

In [7]:
#format to identify ticker and it's associated variable
df['Quarter'] = df['index'].str[-10:-8]
df['Year'] = df['index'].str[-8:-6]
df['Country'] = df['index'].str[-13:-11]
df['Variable'] = df['index'].str[0:4]
df=df.rename(columns={"index":"Ticker","px_last":"Value"})


In [8]:
#rename to identify variables
variable= {'BYXY':'Rates','ECUP':'Unemp','ECCB':'Pol Rate','ECCC':'Core CPI','ECPI':'CPI','ECGQ':'Real GDP'}
#country= {'AU':193,'CA':156,'UK':112,'US':111,'CH':924,'IN':534,'EU':'025','JP':158 }
#df['Country'] = df['Country'].map(country)
df['Variable'] = df['Variable'].map(variable)

In [9]:
#Rename column names and year 
df['Ticker'] = df['Variable'].map(str) +"_" +  df['Country'].map(str)
df['Year'] = "20" + df['Year'].map(str)+ df['Quarter'].map(str)

In [10]:
final=df
final = final[final['Country'].notna()]

In [11]:
final=df
final=final.drop(['Quarter','Country','Variable'],axis=1)
final=final[['Year','Value','Ticker']]
final=final.pivot_table(index='Year',columns='Ticker',values='Value').reset_index()
final=final.replace(np.nan, '', regex=True)
final

Ticker,Year,CPI_AU,CPI_CA,CPI_CN,CPI_EU,CPI_GB,CPI_IN,CPI_JP,CPI_US,Core CPI_US,...,Real GDP_GB,Real GDP_JP,Real GDP_US,Unemp_AU,Unemp_CA,Unemp_CN,Unemp_EU,Unemp_GB,Unemp_JP,Unemp_US
0,2021Q1,1.1,1.4,0.1,1.0,0.8,4.9,-0.4,1.8,1.5,...,-2.0,-1.85,0.2,6.0,9.0,3.8,8.3,5.3,3.0,6.2
1,2021Q2,3.5,3.4,1.3,1.8,1.9,5.69,-0.2,4.8,3.2,...,5.1,7.5,12.7,5.3,7.9,3.93,8.0,4.75,2.9,5.9
2,2021Q3,2.8,4.0,1.1,2.8,2.8,5.17,-0.2,5.2,3.6,...,1.5,2.2,5.2,4.95,7.3,3.86,7.6,4.6,2.9,5.1
3,2021Q4,3.0,4.6,1.85,4.2,4.1,5.08,0.6,5.95,4.0,...,1.1,0.2,5.0,4.8,6.6,3.86,7.65,4.8,2.8,4.5
4,2022Q1,2.95,4.3,1.67,3.1,4.2,5.75,0.4,5.6,3.95,...,1.0,2.5,4.55,4.7,6.4,3.8,7.65,4.7,2.8,4.2
5,2022Q2,2.7,3.7,2.05,2.75,4.3,5.1,1.1,4.0,3.1,...,0.8,2.8,3.9,4.45,6.1,3.7,7.6,4.6,2.7,4.0
6,2022Q3,2.4,2.9,2.4,2.1,3.5,5.22,0.9,3.2,2.6,...,0.6,4.0,4.1,4.3,6.0,3.7,7.5,4.55,2.6,3.8
7,2022Q4,2.3,2.3,2.4,1.6,2.5,5.0,0.7,2.6,2.5,...,0.6,2.8,3.6,4.2,5.9,3.61,7.3,4.5,2.55,3.8
8,2023Q1,2.3,2.05,2.1,1.5,2.5,4.74,0.8,2.45,2.4,...,0.5,2.05,3.1,4.1,5.9,3.61,7.2,4.35,2.5,3.7
9,2023Q2,,,2.2,1.5,2.0,,,2.4,2.35,...,0.4,,2.7,,,3.61,7.2,4.3,,3.65


In [13]:
##Export file to Excel:
final.to_excel('Macro/Bloomberg_Forecast.xlsx',index=False)